### IMPORT MODULES

In [1]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import os
from nibabel.streamlines import save as save_trk
from nibabel.streamlines import Tractogram
from os import path
from dipy.tracking.local import ParticleFilteringTracking
from dipy.core.gradients import gradient_table
from dipy.data import get_sphere, Sphere, HemiSphere
from dipy.direction import (peaks_from_model, ProbabilisticDirectionGetter)
from dipy.io import read_bvals_bvecs
from dipy.io.image import save_nifti
from dipy.io.streamline import save_trk, load_trk
from dipy.reconst.peaks import reshape_peaks_for_visualization
from dipy.reconst.dti import TensorModel, fractional_anisotropy, mean_diffusivity
from dipy.reconst.csdeconv import (ConstrainedSphericalDeconvModel,auto_response, recursive_response, response_from_mask)
from dipy.segment.mask import median_otsu
from dipy.tracking.local import (LocalTracking, ThresholdTissueClassifier, ActTissueClassifier, CmcTissueClassifier)
from dipy.tracking.streamline import Streamlines
from dipy.tracking.utils import random_seeds_from_mask, move_streamlines
from dipy.viz import actor, window
from dipy.tracking.utils import target, density_map

In [2]:
os.chdir('/space/hemera/1/users/cmaffei/hcp_processing/mgh_1015/')
wd = '/space/hemera/1/users/cmaffei/hcp_processing/mgh_1015'

### LOAD DATA

In [ ]:
# Import DWI data, bvec, bval, and binary mask
fimg =  ('10k/data_corr.nii.gz')
img = nib.load(fimg)
data = img.get_data()
print("Data Shape: " + str(data.shape))
affine = img.affine #no need to store the affine tho. 

#Bval and bvec file information import
fbval= ('10k/bvals')
fbvec = ("10k/bvecs")
bvals, bvecs = read_bvals_bvecs(fbval, fbvec)
gtab = gradient_table(bvals,bvecs)
# print("B-Values: \n" + str(gtab.bvecs))
# print(gtab)
#Read the voxel size from the image header.
voxel_size = img.header.get_zooms()[:3]
print ('Voxel Size: ' + str(voxel_size))

#importing binary mask
mask=("mask_er.nii.gz")
img = nib.load(mask)
mask = img.get_data()
#print('mask.shape (%d, %d, %d)' % mask.shape)
print("Mask Shape: " + str(mask.shape))



In [ ]:
# Import DWI data, bvec, bval, and binary mask
fimg =  ('1k/data.nii.gz')
img = nib.load(fimg)
data_1k = img.get_data()
print("Data Shape: " + str(data_1k.shape))
affine = img.affine #no need to store the affine tho. 

#Bval and bvec file information import
fbval= ('1k/bvals')
fbvec = ("1k/bvecs")
bvals, bvecs = read_bvals_bvecs(fbval, fbvec)
gtab_1k = gradient_table(bvals,bvecs)
print("B-Values: \n" + str(gtab.bvecs))
print(gtab)
#Read the voxel size from the image header.
voxel_size = img.header.get_zooms()[:3]
print ('Voxel Size: ' + str(voxel_size))

In [ ]:
data_small=data[50:90,65:75,50:80]
data_1k_small=data_1k[50:90,65:75,50:80]
mask_small=mask[50:90,65:75,50:80]

CREATE BRAIN MASK

In [ ]:
brain_mask_filename = "mask.nii.gz"
if path.exists(brain_mask_filename) and not recompute:
    brain_mask_img = nib.load(brain_mask_filename).get_data()
else:
    recompute = True
    _, brain_mask_img = median_otsu(data, 4, 1)
#     save_nifti(brain_mask_filename, brain_mask_img.astype("uint8"),
#                dwi_img.affine)

## ESTIMATE RESPONSE FUNCTION FOR CONSTRAINED SPHERICAL DECONVOLUTION [Tournier 2007]

In [ ]:
#The auto_response function will calculate FA for an ROI of radius equal to roi_radius in the center of
#the volume and return the response function estimated in that region for the voxels with FA higher than 0.7.
response, ratio = auto_response(gtab, data, roi_radius=10, fa_thr=0.6)

#The response tuple contains two elements. The first is an array with the eigenvalues of the response function and the
#second is the average S0 for this response.
#The tensor generated from the response must be prolate (two smaller eigenvalues should be equal) and look 
#anisotropic with a ratio of second to first eigenvalue of about 0.2. Or in other words, the axial diffusivity of
#this tensor should be around 5 times larger than the radial diffusivity.
print(response)
# print(ratio)

# full_response = np.array([response[0][0], response[0][1],
#                               response[0][2], response[1]])
# np.savetxt('frf_csd_new.txt', full_response)

In [ ]:
response, ratio = response_from_mask(gtab, data, wm_mask)
print(response)

### VISUALIZE RESPONSE FUNCTION

In [ ]:
# Enables/disables interactive visualization
interactive = True

ren = window.Renderer()
evals = response[0]
evecs = np.array([[0, 1, 0], [0, 0, 1], [1, 0, 0]]).T

sphere = get_sphere('symmetric724')
from dipy.sims.voxel import single_tensor_odf
response_odf = single_tensor_odf(sphere.vertices, evals, evecs)
# transform our data from 1D to 4D
response_odf = response_odf[None, None, None, :]
response_actor = actor.odf_slicer(response_odf, sphere=sphere, colormap='plasma')
ren.add(response_actor)

print('Saving illustration as csd_response.png')
window.record(ren, out_path='csd_response.png', size=(200, 200))
if interactive:
    window.show(ren)


## Recursive estimatio of the response function (Tax 2014)

In [ ]:
#importing binary mask
wm_mask=("1k/fa.nii.gz")
img = nib.load(wm_mask)
wm_mask = img.get_data()

In [ ]:
# try recursive response
# tenmodel = TensorModel(gtab)
# tenfit = tenmodel.fit(data, mask)

# FA = fractional_anisotropy(tenfit.evals)
# MD = mean_diffusivity(tenfit.evals)
wm_mask=wm_mask>0.5
# wm_mask = (np.logical_or(FA >= 0.4, (np.logical_and(FA >= 0.3, MD >= 0.0011))))
# nib.save(nib.Nifti1Image(FA.astype('float32'), img.affine, img.header), os.path.join(wd, 'fa.nii.gz'))
# nib.save(nib.Nifti1Image(wm_mask.astype('float32'), img.affine, img.header), os.path.join(wd, 'wm.nii.gz'))

response = recursive_response(gtab, data, mask=wm_mask, sh_order=8,
                              peak_thr=0.03, init_fa=0.08,
                              init_trace=0.0021, iter=20,
                              convergence=0.01, 
                              parallel=True, nbr_processes=4)

In [ ]:
#check the shape of the signal of the response function, which should be like a pancake:
sphere = get_sphere('symmetric724')
response_signal = response.on_sphere(sphere)
# transform our data from 1D to 4D
response_signal = response_signal[None, None, None, :]
response_actor = actor.odf_slicer(
    response_signal, sphere=sphere, colormap='plasma')
interactive = True
ren = window.Renderer()

ren.add(response_actor)
print('Saving illustration as csd_recursive_response.png')
window.record(ren, out_path='csd_recursive_response_def.png', size=(200, 200))
if interactive:
    window.show(ren)

In [ ]:
#Initialize the model
csd_model = ConstrainedSphericalDeconvModel(gtab, response)
sphere = get_sphere('symmetric724')
#relative_peak_threshold : float in [0., 1.] Only peaks greater than ``min + relative_peak_threshold * scale`` are
#kept, where ``min = max(0, odf.min())`` and ``scale = odf.max() - min``. sh_order default=max sh=8
csd_peaks = peaks_from_model(model=csd_model,
                             data=data_small,
                             sphere=sphere,
                             relative_peak_threshold=.5,
                             min_separation_angle=25,
                             mask=mask_small, return_odf=True, return_sh=True)

#save peaks indices
# nib.save(nib.Nifti1Image(csd_peaks.peak_indices, img.affine),
#                  'csd_peaks_indices_new')
# nib.save(nib.Nifti1Image(reshape_peaks_for_vi

In [ ]:
print(data_small.shape)
print(csd_peaks.peak_values.shape)
vals=csd_peaks.peak_values
val=vals[15,10,10]
print(val)
print(val[1])
peak_ratio=val[1]/val[0]
print(peak_ratio)
print(csd_peaks.odf.shape)

In [ ]:
#Initialize the model
csd_model = ConstrainedSphericalDeconvModel(gtab, response)
sphere = get_sphere('symmetric724')
data_small=data[30:90,65:85,40:70]
mask_small=mask[30:90,65:85,40:70]
#relative_peak_threshold : float in [0., 1.] Only peaks greater than ``min + relative_peak_threshold * scale`` are
#kept, where ``min = max(0, odf.min())`` and ``scale = odf.max() - min``. sh_order default=max sh=8
csd_peaks = peaks_from_model(model=csd_model,
                             data=data_small,
                             sphere=sphere,
                             relative_peak_threshold=.5,
                             min_separation_angle=25,
                             mask=mask_small, return_odf=True, return_sh=True)

#save peaks indices
# nib.save(nib.Nifti1Image(csd_peaks.peak_indices, img.affine),
#                  'csd_peaks_indices_new')
# nib.save(nib.Nifti1Image(reshape_peaks_for_visualization(csd_peaks), img.affine),
#              'csd_peaks_new.nii.gz')


#Compute CSD fODF in mrtrix basis for visuakization in mrview
# csd_peaks_mrtrix = peaks_from_model(model=csd_model,
#                              data=data,
#                              sphere=sphere,
#                              relative_peak_threshold=.5,
#                              min_separation_angle=25, sh_basis_type='mrtrix',
#                              mask=mask, return_odf=True, return_sh=True)

#save odf
# nib.save(nib.Nifti1Image(csd_peaks.shm_coeff.astype(np.float32),
#                                   img.affine, img.header), 'odf_recursive_csd.nii.gz')



In [3]:
#loading odfs
sphere = get_sphere('symmetric724')
from dipy.reconst.shm import sh_to_sf
fimg = ("10k/odfs_01_01_body_cc.nii.gz")
img = nib.load(fimg)
odfs = img.get_data()
odfs_tosf = sh_to_sf(odfs, sphere, sh_order=8)

### VISUALIZE FODs

In [9]:
#Enables/disables interactive visualization

fa= ('dtifit_FA.nii.gz')
img = nib.load(fa)
fa=img.get_data()
slf_mask=("rot_files/slf1_oblique_res.nii.gz")
img = nib.load(slf_mask)
slf_mask = img.get_data()
cc_mask=("rot_files/cc_oblique_res.nii.gz")
img = nib.load(cc_mask)
cc_mask = img.get_data()

ren = window.Renderer()
interactive = True
fodf_spheres = actor.odf_slicer(odfs_tosf, sphere=sphere, scale=0.5, norm=False, colormap='jet')
fodf_spheres.display_extent(61, 61, 0, 140, 0, 96)

slice_actor = actor.slicer(fa)
slice_actor.display(slice_actor.shape[0]//2, None, None)
cc_actor = actor.contour_from_roi(slf_mask,
                                        color=[0.7,0,0], opacity=0.2)
slf_actor = actor.contour_from_roi(cc_mask,
                                        color=[0.1,0.6,0.2], opacity=0.2)

ren.set_camera(position=(10, 0, 0))
ren.add(fodf_spheres)
ren.add(slice_actor)
ren.add(slf_actor)
ren.add(cc_actor)

# print('Saving illustration as csd_odfs.png')
# window.record(ren, out_path='csd_fods.png', size=(600, 600))
if interactive:
    window.show(ren)

### VISUALIZE PEAKS

In [ ]:
# window.clear(ren)
# fodf_peaks = actor.peak_slicer(csd_peaks.peak_dirs, csd_peaks.peak_values, colors=None)
# ren.add(fodf_peaks)
# print('Saving illustration as csd_peaks.png')
# window.record(ren, out_path='csd_peaks.png', size=(600, 600))
# if interactive:
#     window.show(ren)

interactive = True
ren = window.Renderer()
# ren.add(fodf_spheres)
ren.add(actor.peak_slicer(
    csd_peaks.peak_dirs, csd_peaks.peak_values, colors=None))

if interactive:
    window.show(ren, size=(900, 900))
else:
    window.record(
        ren, out_path='csd_direction_field.png', size=(900, 900))
    
#to add the odf to the peaks    
fodf_spheres.GetProperty().SetOpacity(0.4)

ren.add(fodf_spheres)

## TRACTOGRAPHY tissue classifier

In [ ]:
fa = nib.load('dtifit_FA.nii.gz').get_data()

tissue_classifier = ThresholdTissueClassifier(fa, 0.1)
# seeds = random_seeds_from_mask(fa > 0.5, seeds_count=2)
streamline_generator = LocalTracking(
   prob_dg, tissue_classifier, seeds, affine=np.eye(4),
    step_size=0.75)

streamlines = Streamlines(streamline_generator)
save_trk("prova_tissue_class.trk",
         streamlines,
         img.affine,
         shape=img.shape[:3], vox_size=img.header.get_zooms()[:3])
print("Number of streamlines: " + str(len(streamlines)))

VISUALIZE STREAMLINES

In [ ]:
ren.clear()
ren.add(actor.line(streamlines))

if interactive:
    window.show(ren, size=(900, 900))
else:
    print('Saving illustration as det_streamlines.png')
    window.record(ren, out_path='det_streamlines.png', size=(900, 900))

## PROBABILISTIC TRACTOGRAPHY

CSD represents each voxel in the data set as a collection of small white matter fibers with different orientations.
The density of fibers along each orientation is known as the Fiber Orientation Distribution (FOD). 
In order to perform probabilistic fiber tracking, we pick a fiber from the FOD at random at each new location along
the streamline. Note: one could use this model to perform deterministic fiber tracking by always tracking along the
directions that have the most fibers.

In [ ]:
#Because the CSD model represents the FOD using the spherical harmonic basis, we can use the from_shcoeff 
#method to create the direction getter. This direction getter will randomly sample directions from the FOD each 
#time the tracking algorithm needs to take another step.

prob_dg = ProbabilisticDirectionGetter.from_shcoeff(odfs, max_angle=45, sphere=sphere)

#Another way is to represent the FOD using a discrete sphere
#This discrete FOD can be used by the ProbabilisticDirectionGetter
#as a PMF for sampling tracing directions. e need to clip the FOD 
#to use it as a PMF because the latter cannot have negative values
#However this way takes more memory, because it samples the PMF and 
#it holds it in memory (that s why we use small sphere instead of default sphere)

# from dipy.data import small_sphere
# odf = gqfit.odf(small_sphere)
# pmf = odf.clip(min=0)
# prob_dg = ProbabilisticDirectionGetter.from_pmf(pmf, max_angle=30, sphere=small_sphere)


In [ ]:
#loading seed mask
seed_mask=("t12fa_n_pve_2_fs_nearest.nii.gz")
imgm = nib.load(seed_mask)
seed_mask = imgm.get_data()
seed_mask = seed_mask > 0.5
print('seed_mask.shape (%d, %d, %d)' % seed_mask.shape)

#defining seeds
#local tracker assumes that the data is sampled on a regular grid. seeds(same space as the affine!). 
seeds = random_seeds_from_mask(seed_mask, seeds_count=2)

## ANATOMICALLY CONSTRAINED TRACTOGRAPHY (ACT)


Anatomically-constrained tractography (ACT) [Smith2012] uses information from anatomical images to determine when the 
tractography stops. The include_map defines when the streamline reached a ‘valid’ stopping region (e.g. gray matter
partial volume estimation (PVE) map) and the exclude_map defines when the streamline reached an ‘invalid’ stopping
region (e.g. corticospinal fluid PVE map). 

In [ ]:
#Load PVE maps
img_pve_gm = nib.load('t12fa_n_pve_1_res.nii.gz')
img_pve_csf = nib.load('t12fa_n_pve_0_res.nii.gz')
img_pve_wm = nib.load('t12fa_n_pve_2_res.nii.gz')
              
# background = np.ones(img_pve_gm.shape)
# background[(img_pve_gm.get_data() +
#             img_pve_wm.get_data() +
#             img_pve_csf.get_data()) > 0] = 0
                       
# include_map = img_pve_gm.get_data()
# include_map[background > 0] = 1
# exclude_map = img_pve_csf.get_data()

# act_classifier = ActTissueClassifier(include_map, exclude_map)

# fig = plt.figure()
# plt.subplot(121)
# plt.xticks([])
# plt.yticks([])
# plt.imshow(include_map[:, :, data.shape[2] // 2].T, cmap='gray', origin='lower',
#            interpolation='nearest')
# plt.subplot(122)
# plt.xticks([])
# plt.yticks([])
# plt.imshow(exclude_map[:, :, data.shape[2] // 2].T, cmap='gray', origin='lower',
#            interpolation='nearest')
# fig.tight_layout()
# fig.savefig('act_maps.png')


# # Maxlen: Maximum number of steps to track from seed. Used to prevent infinite loops. 
# # return_all : bool If true, return all generated streamlines, otherwise only streamlines reaching end points 
# # or exiting the image.
# all_streamlines_act_classifier = LocalTracking(prob_dg,
#                                                act_classifier,
#                                                seeds, 
#                                                affine=np.eye(4), step_size=0.75,
#                                                return_all=True)

# streamlines = Streamlines(all_streamlines_act_classifier)
# save_trk("/space/hemera/1/users/cmaffei/sscilpy_mrtrix_comparison/10000/from_mgh/csd_prob_10npv_act_45angle.trk",
#          streamlines,
#          img.affine,
#          shape=img.shape[:3], vox_size=img.header.get_zooms()[:3])

## CMC (Continuous Map Criterior)
As ACT but instead of considering PEVs as discrete values, it uses to assign weights (Girard et al 2014)

In [ ]:
from dipy.tracking.local import ParticleFilteringTracking
from dipy.tracking.local import CmcTissueClassifier
voxel_size = np.average(img_pve_wm.get_header()['pixdim'][1:4])
step_size = 0.75

cmc_classifier = CmcTissueClassifier.from_pve(img_pve_wm.get_data(),
                                              img_pve_gm.get_data(),
                                              img_pve_csf.get_data(),
                                              step_size=step_size,
                                              average_voxel_size=voxel_size)

all_streamlines_act_classifier = LocalTracking(prob_dg,
                                               cmc_classifier,
                                               seeds, 
                                               affine=np.eye(4), step_size=0.75,
                                               return_all=True)

streamlines = Streamlines(all_streamlines_act_classifier)
save_trk("prova_nuova_pve_45_075.trk",
         streamlines,
         img.affine,
         shape=img.shape[:3], vox_size=img.header.get_zooms()[:3])

In [ ]:
#CST extraction
#load ROIs
img = nib.load ('/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/structural/rois/ic_l_con.nii.gz')
ic = img.get_data()
img =  nib.load ('/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/structural/rois/mb_l_con.nii.gz')
mb = img.get_data()

#load csd tractogram
streamlines_load, hdr = load_trk('/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/10000/from_mgh/csd_prob_npv1_comparisontogqi.trk')
streamlines_load = Streamlines(streamlines_load)
streamlines_load = move_streamlines(streamlines_load, np.eye(4), img.affine)

cst_streamlines_int = target(streamlines_load, ic, affine=np.eye(4))
cst_streamlines = target(cst_streamlines_int, mb, affine=np.eye(4))
cst_csd = Streamlines(cst_streamlines)

#save cst csd
# save_trk('/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/10000/from_mgh/cst_csd_1npv_recursive_prob.trk',
#          cst_csd,
#          img.affine,
#          shape=img.shape[:3], vox_size=img.header.get_zooms()[:3])

#load gqi tractogram
streamlines_load, hdr = load_trk('/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/10000/from_mgh/prob_gqi_odf_norm_pmfthr04.trk')
streamlines_load = Streamlines(streamlines_load)
streamlines_load = move_streamlines(streamlines_load, np.eye(4), img.affine)

cst_streamlines_int = target(streamlines_load, ic, affine=np.eye(4))
cst_streamlines = target(cst_streamlines_int, mb, affine=np.eye(4))
cst_gqi = Streamlines(cst_streamlines)

#save 
# save_trk('/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/10000/from_mgh/cst_gqi_1npv_pmf04_prob.trk',
#           cst_gqi,
#           img.affine,
#           shape=img.shape[:3], vox_size=img.header.get_zooms()[:3])

In [ ]:
from dipy.viz import window, actor
from dipy.viz.colormap import line_colors

# Enables/disables interactive visualization
interactive = True

# Make display objects
cst_csd_streamlines_actor = actor.line(cst_csd, (1., 0.5, 0))
cst_gqi_streamlines_actor = actor.line(cst_gqi, (1., 1., 0), opacity=0.6)
# cc_ROI_actor = actor.contour_from_roi(cc_slice, color=(1., 1., 0.),
#                                      opacity=0.5)

vol_actor = actor.slicer(FA)   
vol_actor.display(x=90)
vol_actor.opacity(0.6)
# Add display objects to canvas
r = window.Renderer()
r.add(cst_csd_streamlines_actor)
r.add(cst_gqi_streamlines_actor)
r.add(vol_actor)

# Save figures
r.set_camera(position=[-1, 0, 0], view_up=[0, 0, 1])
window.record(r, n_frames=1, out_path='/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/10000/from_mgh/cst_comparison.png',
              size=(800, 800))
if interactive:
    window.show(r)